# Fake News Detection Using Machine Learning & NLP

## 📌 Problem Statement
The rapid spread of misinformation through digital media has made fake news detection a critical challenge. This project aims to build a machine learning model that can automatically classify news articles as **Real** or **Fake** based on their textual content.

---

## 🎯 Objective
The objective of this project is to develop an **end-to-end binary text classification system** using **Natural Language Processing (NLP)** techniques and deploy it as an interactive application.

---

## 🧠 Type of Problem
- Supervised Learning  
- Binary Classification  
- NLP-based Text Classification 

In [1]:
import numpy as np 
import pandas as pd 

## Dataset Description
The dataset consists of two CSV files:


* True.csv → Real news articles
* Fake.csv → Fake news articles

## Dataset Preparation Strategy


1. Assign labels to both datasets
2. Merge them into a single DataFrame
3. Shuffle data to avoid ordering bias


In [2]:
true_data = pd.read_csv('True.csv')
true_data['label'] = 1

In [3]:
fake_data = pd.read_csv('Fake.csv')
fake_data['label'] = 0

In [4]:
data = pd.concat([true_data,fake_data],axis=0)

In [5]:
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)


In [7]:
data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

## Why Combine title and text?

Fake news detection depends heavily on **context** and **phrasing**. Using both the title and the article body provides richer semantic information.

In [8]:
data['content'] = data['title']+ " "+ data['text']

In [9]:
data = data[['content','label']]

In [10]:
data.head()

,content,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,0
1,Failed GOP Candidates Remembered In Hilarious...,0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,0
3,California AG pledges to defend birth control ...,1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,0


In [11]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
X = data['content']
y = data['label']

## Train–Test Split

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2,stratify=y)

## Why Use a Pipeline?

Using a pipeline ensures:

1. No data leakage
2. Consistent preprocessing
3. Deployment-ready design
4. Cleaner, reproducible code


In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(
        lowercase=True,
        stop_words='english',
        ngram_range=(1,2),
        max_features=5000
    )),
    ('model', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ))
])

pipe.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 stop_words='english')),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [15]:
from sklearn.metrics import accuracy_score
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.989532293986637


## Why Cross-Validation?

A single train-test split can give misleading results. Cross-validation provides a robust estimate of generalization performance.

In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    pipe,
    X_train,
    y_train,
    cv=5,
    scoring = 'f1'
)

In [17]:
print(scores.mean())

0.987593675598719


In [18]:
import joblib
joblib.dump(pipe,'fake_news_pipeline.pkl')

['fake_news_pipeline.pkl']